In [1]:
import pandas as pd
import os
from dateutil.parser import parse

In [2]:
def convert_volume(vol):
    if pd.isna(vol) or vol in ['-', '']:
        return None  # Return None for NaN values or empty strings
    try:
        if 'K' in vol:
            return float(vol.replace('K', '')) * 1e3
        elif 'M' in vol:
            return float(vol.replace('M', '')) * 1e6
        else:
            return float(vol)
    except ValueError:
        return None  # Return None if the conversion fails

def try_parse_date(date_str, is_uk_format):
    # Enhanced date parsing that defaults to US format if UK format fails
    try:
        return parse(date_str, dayfirst=is_uk_format)
    except ValueError:
        return pd.NaT

def check_date_format(dates):
    # Check multiple dates to better determine the predominant format
    day_first_counts = 0
    sample_size = min(20, len(dates))  # Check up to the first 10 dates or total count if less
    for date in dates[:sample_size]:
        parts = date.split('/')
        if len(parts) == 3:
            if int(parts[0]) > 12:
                day_first_counts += 1
    # If more than half the sample has the day first, assume day-first formatting
    return day_first_counts > sample_size / 2



In [3]:
directory = 'Investing.com'
combined_data = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        data = pd.read_csv(file_path)
        is_uk_format = check_date_format(data['Date'])
        data['Date'] = data['Date'].apply(lambda x: try_parse_date(x, is_uk_format))
        data = data.dropna(subset=['Date'])
        data.set_index('Date', inplace=True)
        
        price_col = 'Price' if 'Price' in data.columns else 'Close'
        volume_col = 'Vol.' if 'Vol.' in data.columns else 'Volume' if 'Volume' in data.columns else None

        if volume_col:
            data[volume_col] = data[volume_col].apply(convert_volume)
            data.rename(columns={volume_col: f'{filename[:-4]}_volume'}, inplace=True)
        
        data.rename(columns={price_col: f'{filename[:-4]}_price'}, inplace=True)
        columns_to_keep = [f'{filename[:-4]}_price'] + ([f'{filename[:-4]}_volume'] if volume_col else [])
        data = data[columns_to_keep]
        
        if combined_data.empty:
            combined_data = data
        else:
            combined_data = pd.concat([combined_data, data], axis=1, join='outer')

# Evaluate columns for NaN proportion and filter
nan_threshold = 0.3  # Filter out columns with more than 30% NaNs
for column in combined_data.columns:
    if combined_data[column].isna().mean() > nan_threshold:
        combined_data.drop(column, axis=1, inplace=True)

combined_data.sort_index(inplace=True)

# Forward fill only up to the last valid data in each column
for column in combined_data.columns:
    combined_data[column] = combined_data[column].ffill().where(combined_data[column].notna())

combined_data.to_pickle('securities.pkl')

In [4]:
combined_data.tail(40)

0GGH Bonds Global_price  0GGH Bonds Global_volume  \
Date                                                            
2024-03-15                    4.685                    5200.0   
2024-03-18                    4.684                    6680.0   
2024-03-19                    4.689                     610.0   
2024-03-20                    4.697                   15310.0   
2024-03-21                    4.702                     350.0   
2024-03-22                    4.709                     670.0   
2024-03-25                    4.705                   13760.0   
2024-03-26                    4.705                    3260.0   
2024-03-27                    4.721                  314900.0   
2024-03-28                    4.722                    2940.0   
2024-03-29                      NaN                       NaN   
2024-04-01                      NaN                       NaN   
2024-04-02                    4.688                   86300.0   
2024-04-03                    4.686                   71570.0   
2024-04-04                    4.700                  270680.0   
2024-04-05                    4.690                    4480.0   
2024-04-08                    4.680                     800.0   
2024-04-09                    4.697                    1120.0   
2024-04-10                    4.671                     860.0   
2024-04-11                    4.657                  220860.0   
2024-04-12                    4.677                   16950.0   
2024-04-15                    4.650                    1590.0   
2024-04-16                    4.636                     650.0   
2024-04-17                    4.644                    2610.0   
2024-04-18                    4.641                    1290.0   
2024-04-19                    4.643                  164110.0   
2024-04-22                    4.645                    1210.0   
2024-04-23                    4.655                  419110.0   
2024-04-24                    4.637                     560.0   
2024-04-25                    4.624                    1950.0   
2024-04-26                    4.641                    5040.0   
2024-04-29                    4.648                     790.0   
2024-04-30                    4.640                    5830.0   
2024-05-01                      NaN                       NaN   
2024-05-02                    4.650                     800.0   
2024-05-03                    4.668                    6110.0   
2024-05-06                    4.678                     620.0   
2024-05-07                    4.692                  107410.0   
2024-05-08                    4.676                    3140.0   
2024-05-09                    4.680                    3450.0   

            EUNK Equities Europe_price  EUNK Equities Europe_volume  \
Date                                                                  
2024-03-15                       76.40                      20130.0   
2024-03-18                       76.17                      40050.0   
2024-03-19                       76.35                      20100.0   
2024-03-20                       76.33                      31530.0   
2024-03-21                       77.06                      22750.0   
2024-03-22                       77.11                      16710.0   
2024-03-25                       77.15                      50550.0   
2024-03-26                       77.29                      39200.0   
2024-03-27                       77.38                      14180.0   
2024-03-28                       77.65                      28320.0   
2024-03-29                         NaN                          NaN   
2024-04-01                         NaN                          NaN   
2024-04-02                       77.01                      67190.0   
2024-04-03                       77.21                      38110.0   
2024-04-04                       77.34                      18840.0   
2024-04-05                       76.66                      51010.0   
202